![title](eod-futures-prices-updates.drawio.png)

![test](images/eod-futures-prices-updates.drawio.png)

In [1]:
import pandas as pd
from hexcast.clients import CMEFTPClient, RedisClient

In [2]:
cme = CMEFTPClient()

### list files

Here are some examples of files that can be retrieved.  `to_redis` is set to False so that they are not registered in the `redis_key`.

In [7]:
listing = cme.ls(
    # redis_key: str = "files:cmegroup:settle",
    to_redis = False,
    files = "*.csv"
)
listing[:5]

['Expiring_Options_UnderlyingProduct_SettlementReport.csv',
 'Expiring_Options_UnderlyingProduct_SettlementReport_20200901.csv',
 'Expiring_Options_UnderlyingProduct_SettlementReport_20200902.csv',
 'Expiring_Options_UnderlyingProduct_SettlementReport_20200903.csv',
 'Expiring_Options_UnderlyingProduct_SettlementReport_20200904.csv']

In [13]:
listing = cme.ls(
    # redis_key: str = "files:cmegroup:settle",
    to_redis = False,
    files = "comex*.zip"
)
listing[:5]

['comex.settle.20220916.s.csv.zip',
 'comex.settle.20220916.s.txt.zip',
 'comex.settle.20220916.s.xml.zip',
 'comex.settle.20220919.s.csv.zip',
 'comex.settle.20220919.s.txt.zip']

### get future settlements

There are 2 convenience files that we use for our core price updates:

In [14]:
listing = cme.ls(
    # redis_key: str = "files:cmegroup:settle",
    to_redis = False,
    files = "*future.csv"
)
listing[:5]

['comex_future.csv', 'nymex_future.csv']

`get_future_settles` retrieves the files and saves them in the S3 cache:

In [ ]:
cme.get_future_settles()

Note that these files are kept in a separate cache `hexight-ds-archives/cme/settle`

In [22]:
pd.read_csv("s3://hexight-ds-archives/cme/settle/comex_future.csv").sample(3)

,PRODUCT SYMBOL,CONTRACT MONTH,CONTRACT YEAR,CONTRACT DAY,CONTRACT,PRODUCT DESCRIPTION,OPEN,HIGH,HIGH AB INDICATOR,LOW,LOW AB INDICATOR,LAST,LAST AB INDICATOR,SETTLE,PT CHG,EST. VOL,PRIOR SETTLE,PRIOR VOL,PRIOR INT,TRADEDATE
377,MGC,12,2023,NaN,MGCZ23,Micro Gold Futures,1742.5,1742.5,NaN,1733.0,NaN,1733.1,B,1737.8000,-24.6,18,1762.40,4.0,40.0,09/23/2022
416,MHG,7,2026,NaN,MHGN26,Micro Copper Futures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.4675,-.1225,0,3.59,NaN,NaN,09/23/2022
329,HRC,8,2024,NaN,HRCQ24,Nymex Steel Futures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,840.0000,-10.00,0,850.00,NaN,NaN,09/23/2022


### clean futures settlements

`clean_future_files` renames and drops columns, and augments the tables with a number of new fields such as `weekday`, `dayofyear`, etc...  

For convenience, the columns are renamed using redis hash keys (python dicts) `renames:CME_SETTLE:future`, and the dropped columns are contained in redis lists `drops:CME_SETTLE:future`.

In [ ]:
cme.clean_future_files()

These daily files are then saved to S3 (as parquet) before being merged into the histories.

Note that these files are kept in a separate cache `hexight-ds-archives/cme/settle`

In [28]:
pd.read_parquet("s3://hexight-ds-archives/cme/settle/nymex_future.pqt").query("product=='CL'").sample(3)

,product,code,year,symbol,desc,open,hi,lo,last,close,pct,volume,prior_settle,prior_vol,prior_int,date,weekday,yearday,yearmonth
2276,CL,F,2022,CLF25,Light Sweet Crude Oil Futures,66.0,66.0,66.0,66.0,65.74,-2.39,22,68.13,4.0,2708.0,2022-09-23,4,266,9
2292,CL,K,2022,CLK26,Light Sweet Crude Oil Futures,NaN,NaN,NaN,NaN,61.36,-1.85,0,63.21,NaN,26.0,2022-09-23,4,266,9
2320,CL,U,2022,CLU28,Light Sweet Crude Oil Futures,NaN,NaN,NaN,NaN,56.44,-1.49,0,57.93,NaN,NaN,2022-09-23,4,266,9


### merging to regular histories

The daily settlements produced up to this point are then merged into history files on a contract-by-contract basis.  In other words, there is one file for the nymex crude oil futures contract `CL`, one file for the cme lumber contract `LB`, one file for the hot rolled coil steel contract comex `HRC` contract, etc...  

Note that these files are kept in a seerate cache `hexight-ds-archives/futures`


For example

In [24]:
pd.read_parquet("s3://hexight-ds-archives/futures/CL.pqt").sample(3)

,date,symbol,open,hi,lo,close,volume,oi,year,weekday,yearday,yearmonth
6386,2006-04-26,CLU09,71.070000,71.070000,71.070000,71.070000,0,0,2006,2,116,4
213760,2002-11-04,CLX04,22.809999,22.809999,22.809999,22.809999,0,37,2002,0,308,11
30415,1998-05-29,CLX98,16.379999,16.690001,16.340000,16.690001,0,0,1998,4,149,5
